# Development Notebook: Visualization of Class Hierarchies
This notebook was added to facilitate development & refactoring of the code base.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import ipycytoscape
import networkx
import inspect
import theano
import theano.tensor as tt

In [3]:
modules = [
    # specify which modules to search:
    theano,
    tt
]

# lookup for module-wise colors
# TODO: use shades of the parent color for sub-sub-modules
COLORS = {
    'theano.compile.function.pfunc': "red",
    'theano.compile.function.types': "red",
    'theano.compile.io': "red",
    'theano.compile.mode': "red",
    'theano.compile.ops': "red",
    'theano.compile.profiling': "red",
    'theano.configparser': "purple",
    'theano.gof.cc': "yellow",
    'theano.gof.fg': "yellow",
    'theano.gof.graph': "yellow",
    'theano.gof.link': "yellow",
    'theano.gof.op': "yellow",
    'theano.gof.params_type': "yellow",
    'theano.gof.type': "yellow",
    'theano.gof.utils': "yellow",
    'theano.gradient': "green",
    'theano.tensor.basic': "blue",
    'theano.tensor.elemwise': "blue",
    'theano.tensor.extra_ops': "blue",
    'theano.tensor.io': "blue",
    'theano.tensor.subtensor': "blue",
    'theano.tensor.type': "blue",
    'theano.tensor.type_other': "blue",
    'theano.tensor.var': "blue",
    'theano.updates': "orange",
}

# build a dictionary of all classes in the specified modules
classes = {}
for module in modules:
    for name, mem in inspect.getmembers(module):
        if inspect.isclass(mem):
            classes[name] = mem
            if not mem.__module__ in COLORS:
                COLORS[mem.__module__] = "pink"

In [4]:
# helper functions for creating a NetworkX 
def add_or_create(G, cls):
    name = cls.__name__
    if name in G:
        return
    color = COLORS.get(cls.__module__, "pink")
    G.add_node(name, name=name, label=name, color=color, tooltip=str(cls.__module__))
    for superclass in cls.__mro__[1:2]:
        add_or_create(G, superclass)
        G.add_edge(superclass.__name__, name)

def make_graph(classes):
    G = networkx.DiGraph()
    for name, cls in classes.items():
        add_or_create(G, cls)
    return G    

In [5]:
G = make_graph(classes)
cso = ipycytoscape.CytoscapeWidget(height=500)
cso.graph.add_graph_from_networkx(G, directed=True)
cso.set_style([
    {
        'selector': 'node',
        'css': {
            #'background-color': '#9dbaea',
            'background-color': 'data(color)',
            'content': 'data(name)',
            'text-valign': 'center',
            'font-size': 10,
            #'color': 'white',
            #'text-outline-width': 2,
            #'text-outline-color': 'green',
            #'background-color': 'green'
        }
    },
    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}},
    {
        'selector': ':selected',
        'css': {
            'background-color': 'black',
            'line-color': 'black',
            'target-arrow-color': 'black',
            'source-arrow-color': 'black',
            'text-outline-color': 'black'
        }
    },
    {'selector': 'edge', 'style': {'width': 2, 'line-color': '#11479e'}},
    {
        'selector': 'edge.directed',
        'style': {
            'curve-style': 'bezier',
            'target-arrow-shape': 'triangle',
            'target-arrow-color': '#11479e'
        }
    },
    {'selector': 'edge.multiple_edges', 'style': {'curve-style': 'bezier'}}
])
cso.layout.height = '1200px'
cso.set_layout(name="dagre", rankDir="LR", nodeDimensionsIncludeLabels=True, spacingFactor=0.7)
cso

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'LR', 'nodeDimensionsIncludeLabels': True, 'spac…